In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce 940MX (0000:03:00.0)
Using Theano backend.


# 3 char models.

## setup

In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
char_size = len(chars) + 1
char_size

86

In [4]:
chars.insert(0,'\0')
''.join(chars)

'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz\x86\xa4\xa6\xa9\xab\xc3'

In [5]:
#map chars to indices and back again.
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_chars = dict((i,c) for i,c in enumerate(chars))

In [6]:
#type(char_indices),indices_chars


In [7]:
idx  = [char_indices[i] for i in text]

In [8]:
len(idx)

600901

In [9]:
#create a list of every 4th character.
cs = 3
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [10]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [11]:
x1.size,x2.size,x3.size

(200297, 200297, 200297)

In [12]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [13]:
y = np.stack(c4_dat[:-2])

In [14]:
y[:4]

array([30, 29,  1, 40])

In [15]:
n_fac = 42

In [16]:
def embedding_input(name,n_in,n_out):
    inp = Input(name=name,dtype="int64",shape=(1,))
    emb = Embedding(input_dim=n_in,output_dim=n_out,input_length=1)(inp)
    return inp,Flatten()(emb)

In [17]:
c1_in,c1 = embedding_input("c1",char_size,n_fac)
c2_in,c2 = embedding_input("c2",char_size,n_fac)
c3_in,c3 = embedding_input("c3",char_size,n_fac)

In [18]:
n_hidden = 256

In [19]:
dense_in = Dense(n_hidden,activation='relu')
c1_hidden = dense_in(c1)
dense_hidden = Dense(n_hidden,activation='tanh',init='identity')
c2_hidden = merge([dense_in(c2),dense_hidden(c1_hidden)])
c3_hidden = merge([dense_in(c3),dense_hidden(c2_hidden)])

In [20]:
dense_out = Dense(char_size,activation='softmax')
c4_out = dense_out(c3_hidden)

In [21]:
model = Model([c1_in,c2_in,c3_in],c4_out)

In [22]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [23]:
model.optimizer.lr=0.000001

In [24]:
char_size

86

In [25]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 42)         3612        c1[0][0]                         
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 42)         3612        c2[0][0]                         
___________________________________________________________________________________________

In [26]:
model.fit([x1,x2,x3],y, batch_size=64,nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 16s - loss: 4.4004    
Epoch 2/4
200297/200297 [==============================] - 15s - loss: 4.2255    
Epoch 3/4
200297/200297 [==============================] - 15s - loss: 3.8896    
Epoch 4/4
200297/200297 [==============================] - 15s - loss: 3.4797    


In [27]:
model.optimizer.lr = 0.001

In [28]:
model.fit([x1,x2,x3],y, batch_size=64,nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 15s - loss: 3.2514    
Epoch 2/4
200297/200297 [==============================] - 15s - loss: 3.1659    
Epoch 3/4
200297/200297 [==============================] - 15s - loss: 3.1312    
Epoch 4/4
200297/200297 [==============================] - 15s - loss: 3.1128    


In [29]:
model.optimizer.lr = 0.000001
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 15s - loss: 3.1001    
Epoch 2/4
200297/200297 [==============================] - 15s - loss: 3.0901    
Epoch 3/4
200297/200297 [==============================] - 15s - loss: 3.0815    
Epoch 4/4
200297/200297 [==============================] - 15s - loss: 3.0737    


In [30]:
model.optimizer.lr = 0.01
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 15s - loss: 3.0662    
Epoch 2/4
200297/200297 [==============================] - 15s - loss: 3.0589    
Epoch 3/4
200297/200297 [==============================] - 15s - loss: 3.0515    
Epoch 4/4
200297/200297 [==============================] - 15s - loss: 3.0441    


# Test Model

In [31]:
import pdb 

In [32]:
def get_next(inp):
    idxs = [char_indices[i] for i in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    #pdb.set_trace()
    return chars[i]

In [34]:
get_next('the')

' '

In [ ]:
np.argmax([[1,2,3,4,5,6,7,8,9,10]])